In [1]:
from flask import Flask, url_for, request, jsonify, redirect, Response
from flask_cors import CORS, cross_origin

import json
import requests

In [11]:
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"
APP_KEY_SAVED_FILE = "res/kakao_msg/app_key"
NAVER_USER_DATA_SAVED_FILE = "res/naver/accout"

In [12]:
# 네이버 계정 정보를 저장하기 위해 최초로 한 번만 동작시키도록 한다.
# 이후로는 또 동작시켜서는 안된다(계정 정보가 날아갈테니까)
import pickle

naver_user_info = dict({
    "smtp_user_id": "사용자계정@naver.com",
    "smtp_user_pw": "비밀번호"
})

f = open(NAVER_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(naver_user_info, f)
f.close()

In [13]:
# KAKAO REST API KEY 정보를 저장하기 위해 최초로 한 번만 동작시키도록 한다.
# 이후로는 또 동작시켜서는 안된다(키값이 날아갈테니까)
import pickle

app_key = ""

f = open(APP_KEY_SAVED_FILE, 'wb')
pickle.dump(app_key, f)
f.close()

In [3]:
# REST APP KEY 저장 잘 되었는지 확인하는 목적
import pickle

f = open(APP_KEY_SAVED_FILE, 'rb')
app_key = pickle.load(f)
f.close()

In [4]:
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [5]:
import os
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.text import MIMEText
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [6]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
        # TLS 보안 설정 및 시작 (보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
        
        # 이메일 전송
        # 메시지 전송시 반드시 as_string()으로 변환하여 전송해야함
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공!!!')
        else:
            print(res)

In [7]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
        # audio, video는 별도 설정 없이 진행하도록 함
        elif key == 'audio':
            print("음원은 지원하지 않습니다!")
        elif key == 'video':
            print("영상은 지원하지 않습니다!")
        # 그 외의 형식
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
                
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment', 
                       filename = os.path.basename(value['filename']))

        # 파일 첨부!
        multi.attach(msg)
        
    return multi

In [ ]:
# Text만 전송하기
smtp_info = dict({
    "smtp_server": "smtp.naver.com",
    "smtp_user_id": "사용자계정@naver.com",
    "smtp_user_pw": "비밀번호",
    "smtp_port": 587
})

title = "가즈아!!!!!!"
content = "날아오르라 주식이여!!!!"
sender = smtp_info['smtp_user_id']
receiver = smtp_info['smtp_user_id']

msg = MIMEText(_text = content, _charset = "utf-8")

msg['Subject'] = title
msg['From'] = sender
msg['To'] = receiver

send_email(smtp_info, msg)

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/spring2pythonRequestWithData', methods = ['GET', 'POST'])
def spring2pythonRequestWithData():
    
    params = json.loads(request.get_data(), encoding='utf-8')
    print("params: ", params)
    
    if len(params) == 0:
        return jsonify("No Parameter")
    
    params_string = ""
    customer_name = ""
    price = ""
    
    ##################### 개조 구간 시작 #####################
    #params_string = ""
    #for key in params.keys():
    #    params_string += 'key: {}, value: {}<br>'.format(key, params[key])
    
    values = []
    for key in params.keys():
        values.append(params[key])

    print("values: ", values)
    customer_name = values[0]
    price = values[1]
    imgPath = values[2]
    
    print("이름: ", customer_name)
    print("가격: ", price)
    print("이미지 경로: ", imgPath)
    
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)
    #print("tokens: ", tokens)

    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers = {
        "Authorization" : "Bearer " + tokens['access_token']
    }
    
    message = "주문 테스트 - 구매자: {0}, 결제 금액: {1}".format(customer_name, price)

    data = {
        "template_object" : json.dumps({
            "object_type" : "text",
            "text" : message,
            "link" : {
                "web_url" : "http://localhost:8080/"
            }
        })
    }

    response = requests.post(url, headers=headers, data=data)
    print(response.status_code)
    
    if response.status_code == 401:
        tokens = update_tokens(app_key, KAKAO_TOKEN_SAVED_FILE)
        
    #print("tokens: ", tokens)
    
    ##################### 개조 구간 끄읕 #####################
    
    ##################### 이메일 전송 시작 #####################
    
    
    ##################### 이메일 전송 끄읕 #####################
    
    return jsonify(params_string)

if __name__ == "__main__":
    app.run()